In [12]:
import torch

In [13]:
import urllib.request

url =(
    "https://raw.githubusercontent.com/rasbt/"
    "LLMs-from-scratch/main/ch05/"
    "01_main-chapter-code/gpt_download.py"
)

file_name = url.split("/")[-1]
urllib.request.urlretrieve(url, file_name)

URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [15]:
from gpt_download import download_and_load_gpt2

settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")


Primary URL (https://openaipublic.blob.core.windows.net/gpt-2/models/124M/checkpoint) failed. Attempting backup URL: https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2/124M/checkpoint


URLError: <urlopen error [Errno 8] nodename nor servname provided, or not known>

In [17]:
# Load settings and params
import tensorflow as tf
import json
import os 

from gpt_download import load_gpt2_params_from_tf_ckpt

model_dir = "../04-implement_training_loop/gpt2/124M"
tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
settings = json.load(open(os.path.join(model_dir, "hparams.json"), "r", encoding="utf-8"))
params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)


In [18]:
config = {
    "vocab_size": 50257,
    "context_size": 1024,
    "embed_dim": 768,
    "num_heads": 12,
    "num_layers": 12,
    "dropout": 0.1,
    "qkv_bias": True,
}

from llm_from_papers.models.gpt import GPT2Model


gpt = GPT2Model(config)
gpt.eval()

GPT2Model(
  (token_embedding): Embedding(50257, 768)
  (position_embedding): Embedding(1024, 768)
  (dropout_embed): Dropout(p=0.1, inplace=False)
  (transfomer_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feed_forward): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (dropout_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAtten

In [24]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shapes must match. Left : {left.shape}, Right: {right.shape}")
    
    return torch.nn.Parameter(torch.tensor(right))

In [33]:
import numpy as np

def load_weights(gpt: GPT2Model, params):
    gpt.position_embedding.weight = assign(
        gpt.position_embedding.weight, params["wpe"])
    gpt.token_embedding.weight = assign(
        gpt.token_embedding.weight, params["wte"])
    
    for b in range(len(params["blocks"])):
        # assign Query, Key, Value weights 
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.transfomer_blocks[b].att.W_query.weight = assign(
            gpt.transfomer_blocks[b].att.W_query.weight, q_w.T)
        gpt.transfomer_blocks[b].att.W_key.weight = assign(
            gpt.transfomer_blocks[b].att.W_key.weight, k_w.T)
        gpt.transfomer_blocks[b].att.W_value.weight = assign(
            gpt.transfomer_blocks[b].att.W_value.weight, v_w.T)
        
        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.transfomer_blocks[b].att.W_query.bias = assign(
            gpt.transfomer_blocks[b].att.W_query.bias, q_b
        )
        gpt.transfomer_blocks[b].att.W_key.bias = assign(
            gpt.transfomer_blocks[b].att.W_key.bias, k_b
        )
        gpt.transfomer_blocks[b].att.W_value.bias = assign(
            gpt.transfomer_blocks[b].att.W_value.bias, v_b
        )

        gpt.transfomer_blocks[b].att.out_proj.weight = assign(
            gpt.transfomer_blocks[b].att.out_proj.weight, params["blocks"][b]["attn"]["c_proj"]["w"].T
        )
        gpt.transfomer_blocks[b].att.out_proj.bias = assign(
            gpt.transfomer_blocks[b].att.out_proj.bias, params["blocks"][b]["attn"]["c_proj"]["b"]
        )


        gpt.transfomer_blocks[b].feed_forward.layers[0].weight = assign(
            gpt.transfomer_blocks[b].feed_forward.layers[0].weight,
            params["blocks"][b]["mlp"]["c_fc"]["w"].T
        )
        gpt.transfomer_blocks[b].feed_forward.layers[0].bias = assign(
            gpt.transfomer_blocks[b].feed_forward.layers[0].bias,
            params["blocks"][b]["mlp"]["c_fc"]["b"]
        )
        gpt.transfomer_blocks[b].feed_forward.layers[2].weight = assign(
            gpt.transfomer_blocks[b].feed_forward.layers[2].weight,
            params["blocks"][b]["mlp"]["c_proj"]["w"].T
        )
        gpt.transfomer_blocks[b].feed_forward.layers[2].bias = assign(
            gpt.transfomer_blocks[b].feed_forward.layers[2].bias,
            params["blocks"][b]["mlp"]["c_proj"]["b"]
        )

        gpt.transfomer_blocks[b].norm1.gamma = assign(
            gpt.transfomer_blocks[b].norm1.gamma,
            params["blocks"][b]['ln_1']["g"]
        )
        gpt.transfomer_blocks[b].norm1.beta = assign(
            gpt.transfomer_blocks[b].norm1.beta,
            params["blocks"][b]['ln_1']["b"]
        )
        gpt.transfomer_blocks[b].norm2.gamma = assign(
            gpt.transfomer_blocks[b].norm2.gamma,
            params["blocks"][b]['ln_2']["g"]
        )
        gpt.transfomer_blocks[b].norm2.beta = assign(
            gpt.transfomer_blocks[b].norm2.beta,
            params["blocks"][b]['ln_2']["b"]
        )

    gpt.final_norm.gamma = assign(gpt.final_norm.gamma,
                                  params["g"])
    gpt.final_norm.beta = assign(gpt.final_norm.beta,
                                  params["b"])

    gpt.lm_head.weight = assign(gpt.lm_head.weight,
                                params["wte"])



        
load_weights(gpt, params)     

In [ ]:
torch.save(
    {
        "model_weights": gpt.state_dict(),
    },
    "gpt2_124M.pth"
)

In [37]:
from llm_from_papers.utils import sample
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [60]:
sentance = "What don't kill us"

tokens = torch.Tensor(tokenizer.encode(sentance)).unsqueeze(0).long()
tokens

tensor([[2061,  836,  470, 1494,  514]])

In [61]:
response = sample(gpt, tokens, 100, context_length=config["context_size"], temperature=1.4, top_k=50)

In [62]:
tokenizer.decode(response[0].numpy().tolist())

'What don\'t kill us and get us off a hill and onto the street to have an emotional support that will help us fight a disease!"\n\nThe event also featured a rally at Liberty Chapel where Dr. Steve Jones spoke out publicly as well as speaking on behalf of the patients that took his place.\n\nHe said he knew as soon as the fundraiser started he would get emails from his supporters that "everyone had a story of Dr. Steve Jones raising funds because he is a survivor." For those who still haven\'t'